In [1]:
# 데이터분석 관련 모듈 로딩
import pandas as pd
import numpy as np


# 모델링 관련 모듈 로딩
# Pytorch 관련
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchinfo import summary
from torchmetrics.classification import BinaryF1Score
import torch.optim.lr_scheduler as lr_scheduler

# scikit-learn 관련
from sklearn.preprocessing import *
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler


# 사용자 생성 모듈
import funcMake
from Cheat_Class import *

# 그 외 기타 모듈 로딩
import os
import pickle

In [2]:
# 저장 경로
PATH = './CheatModels/'

# 모델 파일명
MODEL_FILE = '90_loss(0.28261)_score(0.90108).pth'

funcMake.checkPath(PATH+MODEL_FILE)

경로있음


In [3]:
Cheat_model = torch.load(PATH+MODEL_FILE, weights_only=False)
print(Cheat_model)

MyCheatModel(
  (in_layer): Linear(in_features=2, out_features=250, bias=True)
  (in_hids): ModuleList(
    (0): Linear(in_features=250, out_features=500, bias=True)
    (1): Linear(in_features=500, out_features=250, bias=True)
  )
  (out_layer): Linear(in_features=250, out_features=1, bias=True)
)


In [4]:
summary(Cheat_model)

Layer (type:depth-idx)                   Param #
MyCheatModel                             --
├─Linear: 1-1                            750
├─ModuleList: 1-2                        --
│    └─Linear: 2-1                       125,500
│    └─Linear: 2-2                       125,250
├─Linear: 1-3                            251
Total params: 251,751
Trainable params: 251,751
Non-trainable params: 0

In [5]:
# 예시 데이터 1 (어뷰징 의심)
# * 상품코드: 41 (락토핏 골드 2g x 50포)
# * 가격: 7550 

# 예시 데이터 2 (정상가)
# * 상품코드: 60 (세노비스 트리플러스 50+ 1305mg x 90캡슐)
# * 가격: 33920

In [5]:
# 학습에 사용된 스케일러 로딩
with open('SCALE.pkl', 'rb') as f:
    ss = pickle.load(f)

In [6]:
featureDF=pd.read_csv('feature.csv')
targetDF=pd.read_csv('target.csv')

featureDF.drop('Unnamed: 0', axis=1, inplace=True)
targetDF.drop('Unnamed: 0', axis=1, inplace=True)

X_train=featureDF[:100]
y_train=targetDF[:100]

In [9]:
# 스케일링 진행
X_train_scaled = ss.transform(X_train)

trainDS=CheatDataset(pd.DataFrame(X_train_scaled), y_train)
trainDL=DataLoader(trainDS, shuffle=True)

In [45]:
Cheat_model.eval()
with torch.no_grad():
    pre_list, ans_list, check_list = [], [], []
    # 추론/평가
    for featureTS, targetTS in trainDL:
        pre_val = Cheat_model(featureTS)
        pre = (pre_val > 0.5).int().item()
        ans = targetTS.int().item()
        check = 'O' if pre == ans else 'X'
        # print(pre, ans, check)
        pre_list.append(pre)
        ans_list.append(ans)
        check_list.append(check)
    
evalDF=pd.DataFrame([pre_list, ans_list, check_list], index=['예측','라벨','정답'])
evalDF=evalDF.T
print(evalDF)

   예측 라벨 정답
0   1  0  X
1   0  0  O
2   1  1  O
3   1  1  O
4   1  1  O
.. .. .. ..
95  1  1  O
96  1  1  O
97  1  1  O
98  1  1  O
99  0  0  O

[100 rows x 3 columns]


In [44]:
evalDF['정답'].value_counts()/len(evalDF['정답'])

정답
O    0.88
X    0.12
Name: count, dtype: float64

In [12]:
from sklearn.metrics import classification_report, confusion_matrix

# 예측 및 실제 라벨을 비교하는 평가 함수
def evaluate_model(model, dataloader):
    y_pred, y_true = [], []
    
    model.eval()
    with torch.no_grad():
        for featureTS, targetTS in dataloader:
            
            # 모델의 예측값 (0 또는 1)
            pre_val = model(featureTS)
            pred_label = (pre_val > 0.5).int().numpy()
            y_pred.extend(pred_label)
            y_true.extend(targetTS.numpy())

    # 주요 지표 출력
    print("Classification Report:\n", classification_report(y_true, y_pred))
    print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred))

In [13]:
evaluate_model(Cheat_model, trainDL)

Classification Report:
               precision    recall  f1-score   support

         0.0       1.00      0.54      0.70        26
         1.0       0.86      1.00      0.92        74

    accuracy                           0.88       100
   macro avg       0.93      0.77      0.81       100
weighted avg       0.90      0.88      0.87       100

Confusion Matrix:
 [[14 12]
 [ 0 74]]
